<a href="https://colab.research.google.com/github/butterflylaceration/non-markovian-ai/blob/main/AI_for_Non_Markovianity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qutip
import numpy as np
from os.path import dirname, join as pjoin
import scipy.io as sio
from qutip import *
from qutip.qip.operations import rx, ry, rz, rotation
import scipy.io
import pandas as pd
from itertools import permutations, combinations_with_replacement
import itertools as it
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import tensorflow as tf

In [ ]:
xp = (qutip.basis(2, 0) + (1+0j)*qutip.basis(2, 1)).unit()
xm = (qutip.basis(2, 0) - (1+0j)*qutip.basis(2, 1)).unit()
yp = (qutip.basis(2, 0) + (0+1j)*qutip.basis(2, 1)).unit()
ym = (qutip.basis(2, 0) - (0+1j)*qutip.basis(2, 1)).unit()
zp = (qutip.basis(2, 0) + (0+0j)*qutip.basis(2, 1)).unit()
zm = (qutip.basis(2, 1) + (0+0j)*qutip.basis(2, 1)).unit()
states = [xp, xm, yp, ym, zp, zm]
rhos = [xp*xp.dag(), xm*xm.dag(), yp*yp.dag(), ym*ym.dag(), zp*zp.dag(), zm*zm.dag()]
sigs = [sigmax(), sigmay(), sigmaz(), qeye(2)]



def generateData(W):
  ## put this fn inside the pickle for loop , use raw_data = GenerateData for each W
  ## also write tomos = exp2probs(raw_data, runlabel=None) inside the pickle forloop too
  ## since you need to generate tomos0, tomos1, etcetc. tomos are the features
    '''
    Given a W matrix, generate sample count data
    '''

    # encode states and measurements numerically to index into basis
    labels_i = [k for k in it.product(range(6), range(3),range(6),range(3))]
    c = [[0,1],[2,3],[4,5],[4,5]]  # [[xp,xm],[yp,ym],[zp,zm],[zp,zm]]
    Ls = ['xp','xm','yp','ym','zp','zm']
    Lo = ['x','y','z']
    data = {}
    for s1,o1,s2,o2 in labels_i:
        label = f'{Ls[s1]},{Lo[o1]},{Ls[s2]},{Lo[o2]}'
        counts = {}
        # random number of tot counts around 8000
        #tot = int(8000+(np.random.random()*200-100))
        tot=8000
        for m1,m2 in it.product(range(2),repeat=2):
            # if o1=1,m1=2, c[1]=(2,3)  c[o1][m1]=3
            omega=tensor(rhos[s1].trans(), rhos[c[o1][m1]], rhos[s2].trans(), rhos[c[o2][m2]]).full()
            #omega=tensor(rhos[s1], rhos[c[o1][m1]], rhos[s2], rhos[c[o2][m2]]).full()
            #counts[f'{m1}{m2}']=int(np.real(np.trace(np.matmul(W.full(),omega)))*tot)
            counts[f'{m1}{m2}']=np.real(np.trace(np.matmul(W,omega)))*tot
        totp = counts['00']+counts['01']+counts['10']+counts['11']
        for k in counts:
            counts[k]=counts[k]*tot/totp
        data[label]=counts
    return data
def WToData(W):
  ## also put this in the for loop
    # encode states and measurements numerically to index into basis
    labels_i = [k for k in it.product(range(6), range(3),range(6),range(3))]
    c = [[0,1],[2,3],[4,5],[4,5]]  # [[xp,xm],[yp,ym],[zp,zm],[zp,zm]]
    Ls = ['xp','xm','yp','ym','zp','zm']
    Lo = ['x','y','z']
    data = {}
    for s1,o1,s2,o2 in labels_i:
        label = f'{Ls[s1]},{Lo[o1]},{Ls[s2]},{Lo[o2]}'
        counts = {}
        # random number of tot counts around 8000
        #tot = int(8000+(np.random.random()*200-100))
        tot=8000
        #pac=wtrace(W/4, rhos[s1].trans(), qeye(2), rhos[s2].trans(), qeye(2) )
        for m1,m2 in it.product(range(2),repeat=2):
            # if o1=1,m1=2, c[1]=(2,3)  c[o1][m1]=3
            omega=tensor(rhos[s1].trans(), rhos[c[o1][m1]], rhos[s2].trans(), rhos[c[o2][m2]]).full()
            #omega=tensor(rhos[s1], rhos[c[o1][m1]], rhos[s2], rhos[c[o2][m2]]).full()
            #counts[f'{m1}{m2}']=int(np.real(np.trace(np.matmul(W.full(),omega)))*tot)
            counts[f'{m1}{m2}']=np.real(np.trace(np.matmul(W.full(),omega)))*tot#/pac
        totp = counts['00']+counts['01']+counts['10']+counts['11']
        for k in counts:
            counts[k]=counts[k]*tot/totp
        data[label]=counts
    return data

def exp2probs(raw, runlabel):
    '''
    Conversion utility for Tyler's data to a straight list of probabilies
    Reads in data in form
    {"xp,x,xp,x": {"00": 7917, "01": 13, "10": 31, "11": 29} ...
    and generates tomography data in form
    (0, 0, 0, 0) 0.9908635794743429
    (0, 0, 0, 1) 0.0016270337922403004
    :

    '''
    Km = {'x0':0,'x1':1,'y0':2,'y1':3,'z0':4,'z1':5}
    Ks = {'xp':0,'xm':1,'yp':2,'ym':3,'zp':4,'zm':5}
    labels=[]
    probs=[]

    if runlabel is None:
        data = raw

    else:
        data = raw[runlabel]

    for key in data:
        counts = data[key]
        keys=key.split(',')
        # e.g. keys = ['xp','y','yp','z']
        # e.g. counts looks like {"00": 7917, "01": 13, "10": 31, "11": 29}
        #tot=0
        #for c in counts.values():
        #    tot+=c
        tot = counts['00']+counts['01']+counts['10']+counts['11']





        for l1,l2 in ['00','01','10','11']:
            lab= (Ks[keys[0]], Km[keys[1]+l1], Ks[keys[2]], Km[keys[3]+l2])
            # key = ('xp','y0','yp','z0') -> (0,2,2,4) = lab
            prob = counts[l1+l2]/tot



            labels.append(lab)
            probs.append(prob)
    return pd.DataFrame(probs,index=labels)



In [ ]:
## create a for loop that opens all the pickle files

## example code:
# Open a file and use dump()
# import pickle
# var = locals() ## locals is all the local vars
# g = 1000
# var_holder = {}

# for i in range(g):
#  with open('W_exp_uq'+str(i)+'.pkl', 'rb') as f:
#    string = 'W'+str(i)
#    var_holder[string] = f ## for i = 0 create W0
#    pickle.dump(var[string], f)
#    stroW+str(i)
# locals().update(var_holder)

## You'll have W0, W1, W2, ... W999
